# Tutorial 01: Writing first MNIST Program
By Rahul GAWAI

## Introduction
This program is inspired by [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials)

Python used: 3.6
TensorFlow version: 1.10

## Importing packages

In [96]:
import tensorflow as tf

import sys
import os
import urllib.request
import gzip
import numpy as np


## Download dataset to a local folder.

In [97]:
def _print_download_progress(count, block_size, total_size):
    """
    Function used for printing the download progress.
    Used as a call-back function in maybe_download_and_extract().
    """

    # Percentage completion.
    pct_complete = float(count * block_size) / total_size

    # Limit it because rounding errors may cause it to exceed 100%.
    pct_complete = min(1.0, pct_complete)

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [98]:
def download_img(base_url, filename, download_dir, offset):

    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    file_path = os.path.join(download_dir, filename)

    print("\nDownloading", base_url, "at", download_dir, ".")

    file_path, _ = urllib.request.urlretrieve( base_url, file_path, _print_download_progress)

    with gzip.open(file_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=offset)

    img_size = 28
    img_size_flat = img_size * img_size
    num_channels = 1
    img_shape_full = (img_size, img_size, num_channels)
    images_flat = data.reshape(-1, img_size_flat)
    return images_flat

def download_cls(base_url, filename, download_dir, offset):

    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    file_path = os.path.join(download_dir, filename)

    print("\nDownloading", base_url, "at", download_dir, ".")

    file_path, _ = urllib.request.urlretrieve( base_url, file_path, _print_download_progress)

    with gzip.open(file_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset)
        
    return data

In [103]:
url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

x_train_file = "train-images-idx3-ubyte.gz"
y_train_file = "train-labels-idx1-ubyte.gz"
x_test_file = "t10k-images-idx3-ubyte.gz"
y_test_file = "t10k-labels-idx1-ubyte.gz"

target = "mnist_dataset"

x_train_url = url + x_train_file
y_train_url = url + y_train_file
x_test_url = url + x_test_file
y_test_url = url + y_test_file

print()

#save_path = os.path.join(download_dir, filename)

x_train = download_img(x_train_url, x_train_file, target,16)
print("Training Set", len(x_train))

y_train_cls = download_cls(y_train_url, y_train_file, target,8)
print("Training Set", len(x_train))

x_test = download_img(x_test_url, x_test_file, target,16)
print("Training Set", len(x_train))

y_test_cls = download_cls(y_test_url, y_test_file, target,8)
print("Training Set", len(x_train))




- Download progress: 100.0%
- Download progress: 100.0%
- Download progress: 100.0%
- Download progress: 100.0%

In [104]:
num_train = 55000
num_val = 5000
num_test = 10000

print("Training Set", len(x_train), " images and ", len(y_train_cls), " classes.")

X_train = x_train[0:num_train] / 255.0
Y_train_cls = y_train_cls[0:num_train]

X_val = x_train[num_train:] / 255.0
Y_val_cls = y_train_cls[num_train:]

print("Training Set", len(X_train), " images and ", len(Y_train_cls), " classes.")
print("Validation Set", len(X_val), " images and ", len(Y_val_cls), " classes.")

x_test = x_train[0:num_test] / 255.0
y_test_cls = y_test_cls[0:num_test]


Training Set 60000  images and  8  classes.
Training Set 55000  images and  8  classes.
Validation Set 5000  images and  0  classes.


In [101]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
        
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [102]:
# Get the first images from the test-set.
images = x_test[0:9]

print(len(images))

# Get the true classes for those images.
cls_true = y_test_cls[0:9]

print(len(cls_true))

# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true)

9
8


AssertionError: 

## TF-record

[Referred this implementation to create tf.record](https://github.com/tensorflow/models/blob/master/research/slim/datasets/download_and_convert_mnist.py)

In [ ]:
with tf.python_io.TFRecordWriter(training_filename) as tfrecord_writer:
